In [1]:
import sys
import os
from scipy import linalg
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
sys.path.append("..")
# Enable interactive plotting in Jupyter Notebook

from utils.detection_utils import *

In [2]:
sC = stereoCamera(camera_size={0: (480, 240), 1: (480, 240)},
                      anchor_point={0: (587, 269), 1: (598, 433)},
                      camera_matrix={0: np.array([[2.24579312e+03, 0.00000000e+00, 6.06766474e+02],
                                                  [0.00000000e+00, 3.18225724e+03, 2.87228912e+02],
                                                  [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
                                     1: np.array([[9.17450924e+02, 0.00000000e+00, 5.97492459e+02],
                                                  [0.00000000e+00, 1.08858369e+03, 2.96145751e+02],
                                                  [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])},
                      optimized_camera_matrix={0: np.array([[1.98885152e+03, 0.00000000e+00, 5.83904948e+02],
                                                            [0.00000000e+00, 2.71756632e+03, 3.41261625e+02],
                                                            [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
                                               1: np.array([[9.35319179e+02, 0.00000000e+00, 5.90025655e+02],
                                                            [0.00000000e+00, 1.09136910e+03, 2.97696817e+02],
                                                            [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])},
                      projection_error={0: 0.26768362133770185, 1: 0.29408707559840946},
                      distortion={
                          0: np.array(
                              [[-1.53486495e+00, 1.95803727e+01, 1.63594781e-01, -2.81574724e-02, -1.10093707e+02]]),
                          1: np.array([[0.03667417, 0.10305058, 0.00557331, -0.00655738, -0.16404791]])},
                      stereo_calibration_error={0: 0.6988643727550614},
                      translation_matrix={0: [[-0.60330682], [-0.39384531], [1.07405106]]},
                      rotation_matrix={0: [[0.73971458, 0.1145444, 0.66310023],
                                           [-0.09028238, - 0.95960383, 0.26647622],
                                           [0.66683688, - 0.25698261, - 0.69949161]]}
                      )

In [3]:

vL = videoLoader()
Dt1 = ballDetector()   
Dt2 = ballDetector()
files = os.listdir("../videos")
model = YOLO("../weights/best.pt")
test_file = files[6]

frame_counts = []
ball_coords = []
n_vid = 6

In [ ]:
print(len(files))

for i, file in enumerate(files[n_vid:n_vid+1]):
    vL.load_video(f"../videos/{file}")
    frames = vL[100:-100]
    stereo_coords = [[],[]]
    frame_count = []
    for j, frame in enumerate(frames):
        print(i)
        frame1, frame2 = sC(frame)
        results1 = model(frame1)
        results2 = model(frame2)
       
        coord1 = Dt1(results1, frame1.shape)
        coord2 = Dt2(results2, frame2.shape)
        
        if coord1 is None or coord2 is None:
            continue
        stereo_coords[0].append(coord1)
        stereo_coords[1].append(coord2)
        frame_count.append(j)
    ball_coords.append(stereo_coords)
    frame_counts.append(frame_count)
                

In [ ]:
for i, [ball_coords1, ball_coords2] in enumerate(ball_coords):
    print(len(ball_coords1), len(ball_coords2))
    print(len(frame_counts[i]))    

In [4]:
def DLT(P1, P2, point1, point2):
        A = [point1[1] * P1[2, :] - P1[1, :],
             P1[0, :] - point1[0] * P1[2, :],
             point2[1] * P2[2, :] - P2[1, :],
             P2[0, :] - point2[0] * P2[2, :]
             ]
        A = np.array(A).reshape((4, 4))
        B = A.transpose() @ A
        U, s, Vh = linalg.svd(B, full_matrices=False)

        return Vh[3, 0:3] / Vh[3, 3]

def triangulate(SC, point1, point2):
    RT1 = np.concatenate([np.eye(3), [[0], [0], [0]]], axis=-1)
    RT2 = np.concatenate([SC.conf["rotation_matrix"][0], SC.conf["translation_matrix"][0]], axis=-1)

    P1 = SC.conf["camera_matrix"][0] @ RT1
    P2 = SC.conf["camera_matrix"][1] @ RT2

    coordinate = DLT(P1, P2, point1, point2)
    return coordinate

In [ ]:
coords3d = []

for i, [ball_coords1, ball_coords2] in enumerate(ball_coords):
    c3d = []
    for point1, point2 in zip(ball_coords1, ball_coords2):
        c3d.append(triangulate(sC, point1, point2))
    coords3d.append(c3d)

In [ ]:

# Assuming 'coords' is your numpy array of 3D coordinates
coords = np.array(coords3d[0][:])  # Update this line based on your actual data

# Create line connections between sequential points
connections = [[a, b] for a, b in zip(range(len(coords)-1), range(1, len(coords)))]

# Calculate max range to set equal axes
x_range = [coords[:, 0].min(), coords[:, 0].max()]
y_range = [coords[:, 1].min(), coords[:, 1].max()]
z_range = [coords[:, 2].min(), coords[:, 2].max()]

# Find the maximum range
max_range = max(x_range[1] - x_range[0], y_range[1] - y_range[0], z_range[1] - z_range[0])

# Calculate the mid points for each axis
x_mid = sum(x_range) / 2
y_mid = sum(y_range) / 2
z_mid = sum(z_range) / 2

# Set the range for each axis to be the max range centered around the mid point
x_range = [x_mid - max_range / 2, x_mid + max_range / 2]
y_range = [y_mid - max_range / 2, y_mid + max_range / 2]
z_range = [z_mid - max_range / 2, z_mid + max_range / 2]

# Create the figure object
fig = go.Figure()

# Add lines between connected points
for conn in connections:
    fig.add_trace(go.Scatter3d(
        x=[coords[conn[0], 0], coords[conn[1], 0]],
        y=[coords[conn[0], 1], coords[conn[1], 1]],
        z=[coords[conn[0], 2], coords[conn[1], 2]],
        mode='lines',
        line=dict(color='blue', width=2),
        name='Line'
    ))

# Add markers for each point
fig.add_trace(go.Scatter3d(
    x=coords[:, 0],
    y=coords[:, 1],
    z=coords[:, 2],
    mode='markers',
    marker=dict(size=1, color='red'),
    name='Points'
))

# Update plot appearance
fig.update_layout(
    title="3D Line Plot",
    scene=dict(
        xaxis_title='X Coordinate',
        yaxis_title='Y Coordinate',
        zaxis_title='Z Coordinate',
        xaxis=dict(range=x_range, autorange=False),
        yaxis=dict(range=y_range, autorange=False),
        zaxis=dict(range=z_range, autorange=False),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=1)
    )
)

# Show the plot
fig.show()

In [ ]:
print(files)
for i, file in enumerate(files[n_vid:n_vid+1]):
    print(file)
    vL.load_video(f"../videos/{file}")
    frames = vL[100:-100]
    stereo_coords = [[],[]]
    frame_count = []
    for j, frame in enumerate(frames):
        print(j)
        frame1, frame2 = sC(frame)
        cv2.imshow("frame1", frame1)
        cv2.imshow("frame2", frame2)
        cv2.waitKey(0)
    cv2.destroyAllWindows()

# testing stereo vision results


In [ ]:
file = "WhatsApp Video 2024-03-29 at 19.14.14(2).mp4"
vL.load_video(f"../videos/{file}")
frames = vL[100:-100]
for j, frame in enumerate(frames):
    print(j)
    frame1, frame2 = sC(frame)
    cv2.imshow("frame1", frame1)
    cv2.imshow("frame2", frame2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

100
200
300
400
500
